In [3]:
pwd
set more off
clear all


C:\Users\wue04\Box Sync\fresh




### clean up raw distance output from ArcGIS

In [ ]:
forvalues i=12/16 {
    import delimited raw-output\closest-facility\closest_20`i'.csv, clear
    keep facilityrank incidentid total name 
    rename facilityrank rank
    rename total dist
    rename incidentid id
    split name, p(" - ")
    drop name name1
    rename name2 name
    reshape wide dist name, i(id) j(rank)
    gen year=20`i'
    compress
    save data\dist`i'.dta, replace
}
.

In [ ]:
forvalues i=12/15 {
    append using data\dist`i'.dta
    erase data\dist`i'.dta
}
.

merge 1:m id year using data\address\xy12-16.dta
tab year

In [ ]:
drop id _merge
order x y year
sort x y year
compress
save data\distance_xy_allyears.dta, replace
erase data\dist16.dta

### link xy distance to newid

In [ ]:
* clean up AP geocoded files
forvalues i=12/16 {
    use "S:\Restricted Data\Geocoding\AP\newid ap coordinates 20`i'.dta", clear
    keep newid year xcoord ycoord *boro* *zip* *atitude* *ongitude*
    rename xcoord x
    rename ycoord y
    rename *boro* boro
    rename *zip* zipcode
    rename *atitude* lat
    rename *ongitude* lon
    destring x y boro zipcode lat lon, replace
    compress
    save S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta, replace
}
.

In [ ]:
forvalues i=12/15 {
    append using S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta
    erase S:\Personal\hw1220\fresh\data\newid_xy_`i'.dta
}
.

In [52]:
tab year
duplicates tag newid year, gen(dup)
tab dup

merge m:1 x y year using data\distance_xy_allyears.dta

drop _merge dup
drop if missing(newid)|missing(year)
sort newid year

duplicates tag newid year, gen(dup)
tab dup
drop dup

drop if newid=="."
compress
save S:\Personal\hw1220\fresh\data\distance_newid.dta, replace
erase S:\Personal\hw1220\fresh\data\newid_xy_16.dta




Duplicates in terms of newid year


        dup |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |  5,591,788      100.00      100.00
------------+-----------------------------------
      Total |  5,591,788      100.00


### tab store coverage by year

In [5]:
use S:\Personal\hw1220\fresh\data\distance_newid.dta, clear

In [9]:
ds

newid    year     x        y        boro     zipcode  lat      lon      dist1    name1    dist2    name2    dist3    name3    dist4    name4


In [6]:
tab year


Sheet1$_yea |
          r |      Freq.     Percent        Cum.
------------+-----------------------------------
       2012 |  1,054,760       18.86       18.86
       2013 |  1,093,100       19.55       38.41
       2014 |  1,145,820       20.49       58.90
       2015 |  1,164,178       20.82       79.72
       2016 |  1,133,929       20.28      100.00
------------+-----------------------------------
      Total |  5,591,787      100.00


In [8]:
bys year: tab name1 if dist1<=528

In [13]:
tabout name1 if dist1<=528 & year==2016 using test.csv, replace


Table output written to: test.csv

1 name  No.
1635 Lexington Ave      288.0
1851 Bruckner Blvd      29.0
2050 Webster Avenue     279.0
21 Manhattan Avenue     266.0
238 E 161st Street      117.0
300 Sand Lane   47.0
305-325 Avenue M        103.0
34-20 Junction Boulevard        242.0
3470 Third Avenue       363.0
445 East 163rd Street   98.0
459 East 149th Street   51.0
Total   1,883.0


In [1]:
help tabout